Inserire il path all'interno del quale è contenuto il file .ipynb

In [1]:
path = "/content/drive/MyDrive/CODICE FINALE/"

Unzip del database

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Inserire il path all'interno del quale è contenuto il construction set

In [ ]:
!unzip "/content/drive/MyDrive/OCTchallenge_new.zip" # inserire path completa in cui si trovano i dati

# Installazione di MONAI ed esportazione delle librerie

In [4]:
!pip install 'monai[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 KB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [5]:
!pip install pynrrd

import os
from monai.inferers import SimpleInferer
import nrrd

from monai.utils import set_determinism
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.config import print_config
from monai.metrics import DiceMetric,ConfusionMatrixMetric

import monai
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import torch 
import PIL
from PIL import ImageEnhance

from tqdm import tqdm
from skimage import measure, segmentation
from skimage.io import imread, imshow, imsave
from skimage.transform import resize

from skimage.transform import rotate
from scipy.ndimage import binary_fill_holes
from sklearn.preprocessing import minmax_scale

import skimage.io as io
from skimage.measure import label, regionprops, regionprops_table
from skimage import morphology,exposure
import cv2
import pandas as pd

import torch
from torchvision import transforms
from PIL import Image, ImageFilter, ImageEnhance

from monai.transforms import (
    AsDiscrete,
    DataStatsd,
    AddChanneld,
    Compose,
    Activations,
    LoadImage,
    LoadImaged,
    Resize,
    Resized,
    RandFlipd,
    ScaleIntensityRange,
    ScaleIntensityRanged,
    DataStats,
    AsChannelFirstd,
    AsDiscreted,
    ToTensord,
    EnsureType,
    ThresholdIntensityd,
    SpatialCropd,
    CropForegroundd,
    EnsureChannelFirstd,
    RandSpatialCropSamplesd,
    Orientationd,
    Spacingd,
    DataStatsd,
    RandRotate90d,
    RandRotated,
    RandAxisFlipd,
)

from monai.data import (
    DataLoader,
    CacheDataset,
    PILReader,
    ITKReader,
    NrrdReader,
    IterableDataset,
    decollate_batch,
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Definizione dei path

In [6]:
current_dir = os.getcwd()
dataset_name = 'new dataset'

#Poichè all'interno delle cartelle di training e validation vi sono due sotto-cartelle (w6_d5 e w6_d7), 
#sono state create rispettivamente due liste ordinate (sia per le immagini che per le maschere) per training e validation.

training1_volumes_path = os.path.join(dataset_name,'Volumes','training','w6_d5')
training2_volumes_path = os.path.join(dataset_name,'Volumes','training','w6_d7')

validation1_volumes_path = os.path.join(dataset_name,'Volumes','validation','w2_d7')
validation2_volumes_path = os.path.join(dataset_name,'Volumes','validation','w6_d9')

training1_masks_path = os.path.join(dataset_name,'masks','training','w6_d5')
training2_masks_path = os.path.join(dataset_name,'masks','training','w6_d7')

validation1_masks_path = os.path.join(dataset_name,'masks','validation','w2_d7')
validation2_masks_path = os.path.join(dataset_name,'masks','validation','w6_d9')

#liste ordinate per le immagini di training --> 4608 elementi
training1_volumes = sorted(os.listdir(training1_volumes_path))
training1_volumes = training1_volumes[1:] #al fine di eliminare il primo elemento che è ".DS_Store", ovvero un file nascosto con un formato proprietario creato da macOS 
training2_volumes = sorted(os.listdir(training2_volumes_path))

#liste ordinate per le maschere di training
training1_masks = sorted(os.listdir(training1_masks_path))
training2_masks = sorted(os.listdir(training2_masks_path))

#liste ordinate per le immagini di validazione --> 1024 elementi 
validation1_volumes = sorted(os.listdir(validation1_volumes_path))
validation2_volumes = sorted(os.listdir(validation2_volumes_path))


#liste ordinate per le maschere di validazione
validation1_masks = sorted(os.listdir(validation1_masks_path))
validation2_masks = sorted(os.listdir(validation2_masks_path))


# Creazione del training e validation set dict

Creazione del training set dict

In [7]:
training_data = [] # Inizializzazione della lista di tris di immagini di training
for i in range(len(training1_volumes)): #ciclo sul primo set di immagini di training 
    tempDict = {
        'image': os.path.join(training1_volumes_path, training1_volumes[i]),
        'segmentation': os.path.join(training1_masks_path, training1_masks[i])}
    
    training_data.append(tempDict)

for i in range(len(training2_volumes)): #ciclo sul secondo set di immagini di training

    tempDict = {
        'image': os.path.join(training2_volumes_path, training2_volumes[i]),
        'segmentation': os.path.join(training2_masks_path,training2_masks[i])}
    
    training_data.append(tempDict)

Creazione del validation set dict

In [8]:
validation_data = [] # Inizializzazione della lista di tris di immagini di validation
for i in range(len(validation1_volumes)): #ciclo sul primo set di immagini di validazione

    tempDict = {
        'image': os.path.join(validation1_volumes_path, validation1_volumes[i]),
        'segmentation': os.path.join(validation1_masks_path, validation1_masks[i])}
    
    validation_data.append(tempDict)

for i in range(len(validation2_volumes)): #ciclo sul secondo set di immagini di validazione
    tempDict = {
        'image': os.path.join(validation2_volumes_path, validation2_volumes[i]),
        'segmentation': os.path.join(validation2_masks_path,validation2_masks[i])}
    
    validation_data.append(tempDict)

# Pre- e Post- processing

Definizione di una funzione di pre processing

x: immagine 2D originale
y: maschera manuale associata all'immagine x



In [9]:
from scipy.ndimage import median_filter
from scipy.ndimage import gaussian_filter
from skimage import io
from skimage.exposure import rescale_intensity

def preprocessing(x,y): #x:immagine, y:segmentazione

  #Eliminazione del canale --> (256,256)
  x = x.squeeze()
  y = y.squeeze()

  #Conversione delle immagini in formato numpy
  x = x.cpu().numpy()
  y = y.cpu().numpy()

  return x,y

Definizione di una funzione di post processing

x: maschera automatica in output al modello
y: maschera manuale associata all'immagine z
z: immagine 2D pre-processata

In [10]:
def postprocessing(x,y,z):

  #Conversione delle immagini in formato numpy
  x = x.cpu().numpy() 
  y = y.cpu().numpy()
  z = z.cpu().numpy()

  return x,y,z

# Creazione dei Dataset e DataLoader per le fasi di Training e Validation

In [11]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image","segmentation"],image_only=False, reader=ITKReader()), # caricamento dell'immagine e della segmentazione
        EnsureChannelFirstd(keys=["image","segmentation"]), # aggiunge il canale all' immagine e alla segmentazione ottenendo il formato channel-first da (NxN) a (1xNxN)
        Resized(keys=["image", "segmentation"], spatial_size=[256,256]), # resize dell'immagine a 256x256
        AsDiscreted(keys=["segmentation"],threshold=0.5), # porta la segmentazione in formato binario con soglia 0.5
     
        # Operazioni di data augmentation: 
        RandRotated(keys=["image", "segmentation"], range_x=[-0.1, 0.1], prob=0.3, padding_mode="zeros"),
    
        #DataStatsd(keys = ['image', 'segmentation']), #Per verificare le dimensioni finali dell'immagine
        ToTensord(keys=["image", "segmentation"]), # per portare immagini e segmentazioni da formato PIL a un torch tensor da dare in input alla rete
    ]
)

val_transforms = Compose(
    [
        LoadImaged(keys=["image","segmentation"],image_only=False, reader=ITKReader()), # caricamento dell'immagine e della segmentazione
        EnsureChannelFirstd(keys=["image","segmentation"]), # aggiunge il canale all' immagine e alla segmentazione ottenendo il formato channel-first da (NxN) a (1xNxN)
        Resized(keys=["image", "segmentation"], spatial_size=[256,256]), # resize dell'immagine a 256x256
        AsDiscreted(keys=["segmentation"],threshold=0.5), # porta la segmentazione in formato binario con soglia 0.5 (abbiamo modificato il range di valori nella trasformazione precedente)
        ToTensord(keys=["image", "segmentation"]), # per portare immagini e segmentazioni da formato PIL a un torch tensor da dare in input alla rete
    ]
)

In [12]:
batch_size= 32

# creazione di un oggetto iterabile da cui possiamo estrarre in maniera sequenziale dei dati
#  in input forniamo le coppie immagine-segmentazione e le trasformazioni da applicare
train_ds = IterableDataset(data = training_data, transform = train_transforms)

wks = 0 # definisce il numero di workers (core della CPU da utilizzare) per caricare le immagini

# definizione dell'oggetto Dataloader che si occuperà di caricare le immagini in maniera sequenziale
# definendo i vari batch durante l'allenamento. Definiamo ora la Batch Size da utilizzare
train_loader = DataLoader(train_ds, batch_size=batch_size, num_workers=wks, pin_memory=True, shuffle=False)

#Facciamo lo stesso per il validation
val_ds = IterableDataset(data = validation_data, transform = val_transforms)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=wks, pin_memory=True, shuffle=False)

# ANALISI TRAIN

Codice per generare un **dizionario** a partire da dati del **training set**. Da qui si possono ricavare informazioni come l'istogramma dei singoli tumori, la loro area, densità ecc.

In [13]:
eval_num = int(len(training_data)/batch_size) # definire numero di iterazioni per effettuare validazione ogni epoca 
max_iterations = eval_num # numero massimo di batch da iterare
global_step = 1

x_3D= ()
y_3D=()

epoch_iterator = tqdm(train_loader, desc="Pre-processing (X / X Steps)", dynamic_ncols=True, total=eval_num, position=0, leave=True) # crea un oggetto tqdm per tracciare l'avenzamento delle epoche

# Inizializza un dizionario per memorizzare le aree e le densità dei tumori
tumor = {}

# Contatore per le chiavi univoche
counter = 1
slice_=1

for step, batch in enumerate(epoch_iterator): # ad ogni ciclo for estrae una coppia step (numero dell'iterazione), batch (immagine e segmentazione) dal train loader
      
    x1, y1 = (batch["image"], batch["segmentation"]) # manda i tensori di immagine e maschera alla GPU --> (32,1,256,256)

    for i in range(batch_size):

      x2,y2 = x1[i,:,:,:], y1[i,:,:,:]  #Estrazione del primo elemento del batch --> Metatensor (1,256,256)

      #Applicazione della funzione pre-processing
      x2,y2 = preprocessing(x2,y2) #--> x2/y2.size (256,256)
      
      image_name = batch['image_meta_dict']['filename_or_obj'][-1][:-5] # Info sull'immagine e la slice
      image_mask= x2*y2 #Maschera sovrapposta all'immagine
      regions = measure.regionprops(label(y2),image_mask) # Calcola le proprietà delle regioni
    
      for region in regions: # Per ogni regione (tumore), calcola
          area = region.area #area

          tumor_coords = region.coords # Estrae le coordinate dei pixel all'interno della regione del tumore          
          tumor_norm = minmax_scale(x2[tumor_coords[:, 0], tumor_coords[:, 1]], feature_range=(0, 255)) # Normalizza solo i pixel del tumore
          hist, bins = np.histogram(tumor_norm, bins=256, range=(0, 255)) # Calcola l'istogramma della regione normalizzata del tumore
          
          intensity_mean = np.mean(image_mask[region.coords[:,0], region.coords[:,1]])

          tumor[counter] = {'filename': image_name, 'slice': slice_,'area': round(area,1),'intensità media':intensity_mean,'hist': hist }
          counter += 1 # Incrementa il contatore

      if slice_ ==256:
        slice_=0
        
      slice_ +=1 
        
    epoch_iterator.set_description("Pre-processing Train (%d / %d Steps)" % (global_step, max_iterations)) # aggiorna il counter tqdm
    global_step += 1


Pre-processing Train (288 / 288 Steps): 100%|██████████| 288/288 [04:31<00:00,  1.06it/s]


# ANALISI VALIDATION

Codice per generare un **dizionario** a partire dai dati del **validation set** . Da qui si possono ricavare informazioni come l'istogramma dei singoli tumori, la loro area, densità ecc.

In [14]:
eval_num = int(len(validation_data)/batch_size) # definire numero di iterazioni per effettuare validazione ogni epoca 
max_iterations = eval_num # numero massimo di batch da iterare
global_step = 1

epoch_iterator = tqdm(val_loader, desc="Pre-processing (X / X Steps)", dynamic_ncols=True, total=eval_num, position=0, leave=True) # crea un oggetto tqdm per tracciare l'avenzamento delle epoche

# Inizializza un dizionario per memorizzare le aree e le densità dei tumori
tumor_val = {}

# Contatore per le chiavi univoche
counter = 1
slice_=1

for step, batch in enumerate(epoch_iterator): # ad ogni ciclo for estrae una coppia step (numero dell'iterazione), batch (immagine e segmentazione) dal train loader
      
    x1, y1 = (batch["image"], batch["segmentation"]) # manda i tensori di immagine e maschera alla GPU --> (32,1,256,256)

    for i in range(batch_size):

      x2,y2 = x1[i,:,:,:], y1[i,:,:,:]  #Estrazione del primo elemento del batch --> Metatensor (1,256,256)

      #Applicazione della funzione pre-processing
      x2,y2 = preprocessing(x2,y2) #--> x2/y2.size (256,256)

      image_name = batch['image_meta_dict']['filename_or_obj'][-1][-18:-8] # Info sull'immagine e la slice
      image_mask= x2*y2 #Maschera sovrapposta all'immagine
      regions = measure.regionprops(label(y2),image_mask) # Calcola le proprietà delle regioni

      for region in regions: # Per ogni regione (tumore), calcola l'area e la densità
          area = region.area #Area
          
          tumor_coords = region.coords # Estrae le coordinate dei pixel all'interno della regione del tumore          
          tumor_norm = minmax_scale(x2[tumor_coords[:, 0], tumor_coords[:, 1]], feature_range=(0, 255)) # Normalizza solo i pixel del tumore
          hist, bins = np.histogram(tumor_norm, bins=256, range=(0, 255)) # Calcola l'istogramma della regione normalizzata del tumore
          
          intensity_mean = np.mean(image_mask[region.coords[:,0], region.coords[:,1]])

          tumor_val[counter] = {'filename': image_name, 'slice': slice_,'area': round(area,1),'intensità media':intensity_mean,'hist': hist }
          counter += 1 # Incrementa il contatore

      if slice_ ==256:
        slice_=0

      slice_ +=1 
        
    epoch_iterator.set_description("Pre-processing Val (%d / %d Steps)" % (global_step, max_iterations)) # aggiorna il counter tqdm
    global_step += 1

Pre-processing Val (64 / 64 Steps): 100%|██████████| 64/64 [00:52<00:00,  1.21it/s]


# SALVATAGGIO/CARICAMENTO DEI DIZIONARI

SALVATAGGIO DEL DIZIONARIO

In [15]:
import pickle

with open("tumor.pickle", "wb") as f:
  pickle.dump(tumor, f)

with open("tumor_val.pickle", "wb") as f:
  pickle.dump(tumor_val, f)

LOAD DEL DIZIONARIO

In [ ]:
import pickle

with open(path + "tumor.pickle", "rb") as f:
  tumor = pickle.load(f)

with open(path + "tumor_val.pickle", "rb") as f:
  tumor_val = pickle.load(f)

# TABELLA RIASSUNTIVA

In [17]:
# Creazione dei DataFrame di Pandas
df_train = pd.DataFrame.from_dict(tumor, orient='index')
df_val = pd.DataFrame.from_dict(tumor_val, orient='index')

# Aggiunta della colonna "dataset"
df_train['dataset'] = 'training set'
df_val['dataset'] = 'validation set'

# Unione dei due DataFrame
df = pd.concat([df_train, df_val])

# Raggruppamento per dataset
df_grouped = df.groupby('dataset')

# Calcolo delle statistiche per ogni gruppo
df_stats = pd.DataFrame({
    'area minima': df_grouped['area'].min(),
    'area massima': df_grouped['area'].max(),
    'intensità minima': df_grouped['intensità media'].min(),
    'intensità massima': df_grouped['intensità media'].max()
})

# Visualizzazione del DataFrame
print(df_stats)

                area minima  area massima  intensità minima  intensità massima
dataset                                                                       
training set              1          6363        323.322357       12235.237305
validation set            1         10336        416.000000       17295.644531
